In [1]:
import torch
import os

device = "cuda" if torch.cuda.is_available() else "cpu"

with_storage_dbscan = False
os.makedirs("optuna_storage", exist_ok=True)
dbscan_optuna_storage_path = "sqlite:///optuna_storage/dbscan_study.db"

train_set_path = (
    "/home/jbct/Projects/thesis/db-ocsvm/data/processed/NSL-KDD/train_set_full.csv"
)
test_set_path = (
    "/home/jbct/Projects/thesis/db-ocsvm/data/processed/NSL-KDD/test_set.csv"
)

results_path = "tuning_results/results_01.json"

test_run = True

if test_run:
    sample_size = 0.03
    use_sample = True
else:
    sample_size = 1.0
    use_sample = False


use_full_train_set = True

dbscan_tuning_parameters = {
    "evaluation_metric": "silhouette",  # silhouette, calinski_harabasz, davies_bouldin
    "distance_metric": "manhattan",  # manhattan, euclidean
    "trials": 10,
}

existing_model_path = "best_models/config 5/autoencoder_Model_1_hidden[96, 64]_latent55_lr0.001_bs128_optadamw_actELU_dr0.1_wd0.pth"

existing_model_architecture = {
    "input_dim": 122,
    "hidden_dims": [96, 64],
    "latent_dim": 55,
    "activation_type": "ELU",
    "negative_slope": 0.02,
    "dropout_rate": 0.1,
    "output_activation_type": "Sigmoid",
}

import dataset

In [2]:
import pandas as pd

train_df = pd.read_csv(train_set_path)

if use_sample:
    train_df = train_df.sample(frac=sample_size, random_state=42).reset_index(drop=True)

print(train_df.shape)
train_df.head(1)

(2020, 122)


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0.0,5.833486e-07,2.572642e-07,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [3]:
from sklearn.model_selection import train_test_split

X_train_full = train_df.values

X_train, X_val = train_test_split(train_df, test_size=0.2, random_state=42)
X_train = X_train.values
X_val = X_val.values

print(X_train.shape, X_val.shape, X_train_full.shape)

(1616, 122) (404, 122) (2020, 122)


In [4]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train)
X_val_tensor = torch.FloatTensor(X_val)

# Create data loaders
train_dataset = TensorDataset(X_train_tensor)
val_dataset = TensorDataset(X_val_tensor)

input_dim = X_train_full.shape[1]

training the autoencoder or use existing

In [5]:
import torch.nn as nn
import torch.nn as nn
from typing import List, Optional


class BatchNormAutoencoderV2(nn.Module):
    def __init__(
        self,
        input_dim: int = 128,
        hidden_dims: List[int] = [64, 32],
        latent_dim: int = 16,
        activation_type: str = "ReLU",
        negative_slope: float = 0.2,
        dropout_rate: float = 0.2,  # Added dropout
        output_activation_type: Optional[str] = "Sigmoid",  # Default for [0,1] data
    ) -> None:
        super(BatchNormAutoencoderV2, self).__init__()

        # Select activation function
        activation: nn.Module
        if activation_type == "ReLU":
            activation = nn.ReLU()
        elif activation_type == "LeakyReLU":
            activation = nn.LeakyReLU(negative_slope)  # Better negative gradient
        elif activation_type == "ELU":
            activation = nn.ELU()
        elif activation_type == "GELU":  # Adding GELU option
            activation = nn.GELU()
        else:
            raise ValueError("Unknown activation type provided")

        # Build encoder
        encoder_layers: List[nn.Module] = []
        current_dim: int = input_dim
        for h_dim in hidden_dims:
            encoder_layers.append(nn.Linear(current_dim, h_dim))
            encoder_layers.append(nn.BatchNorm1d(h_dim))
            encoder_layers.append(activation)
            encoder_layers.append(nn.Dropout(dropout_rate))  # Add dropout
            current_dim = h_dim

        # Latent layer
        encoder_layers.append(nn.Linear(current_dim, latent_dim))
        self.encoder: nn.Sequential = nn.Sequential(*encoder_layers)

        # Select output activation function
        output_activation: Optional[nn.Module] = None
        if output_activation_type == "ReLU":
            output_activation = nn.ReLU()
        elif output_activation_type == "LeakyReLU":
            output_activation = nn.LeakyReLU(0.2)
        elif output_activation_type == "ELU":
            output_activation = nn.ELU()
        elif output_activation_type == "Sigmoid":
            output_activation = nn.Sigmoid()
        elif output_activation_type == "Tanh":
            output_activation = nn.Tanh()
        elif output_activation_type is None:
            output_activation = None
        else:
            raise ValueError("Unknown activation type provided")

        # Build decoder
        decoder_layers: List[nn.Module] = []
        current_dim = latent_dim
        for h_dim in reversed(hidden_dims):
            decoder_layers.append(nn.Linear(current_dim, h_dim))
            decoder_layers.append(nn.BatchNorm1d(h_dim))
            decoder_layers.append(activation)
            decoder_layers.append(nn.Dropout(dropout_rate))  # Add dropout
            current_dim = h_dim

        # Add final output layer (no batch norm on output layer)
        decoder_layers.append(nn.Linear(current_dim, input_dim))

        # Add output activation if specified
        if output_activation is not None:
            decoder_layers.append(output_activation)

        self.decoder: nn.Sequential = nn.Sequential(*decoder_layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        encoded: torch.Tensor = self.encoder(x)
        decoded: torch.Tensor = self.decoder(encoded)
        return decoded

    def encode(self, x: torch.Tensor) -> torch.Tensor:
        return self.encoder(x)

In [6]:
from torch import nn

autoencoder = BatchNormAutoencoderV2(
    input_dim=existing_model_architecture["input_dim"],
    hidden_dims=existing_model_architecture["hidden_dims"],
    latent_dim=existing_model_architecture["latent_dim"],
    activation_type=existing_model_architecture["activation_type"],
    negative_slope=existing_model_architecture["negative_slope"],
    dropout_rate=existing_model_architecture["dropout_rate"],
    output_activation_type=existing_model_architecture["output_activation_type"],
)

In [7]:
# Load best model
checkpoint = torch.load(existing_model_path)
autoencoder.load_state_dict(checkpoint["model_state_dict"])

autoencoder.to(device)
autoencoder.eval()

BatchNormAutoencoderV2(
  (encoder): Sequential(
    (0): Linear(in_features=122, out_features=96, bias=True)
    (1): BatchNorm1d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0)
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=96, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ELU(alpha=1.0)
    (7): Dropout(p=0.1, inplace=False)
    (8): Linear(in_features=64, out_features=55, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=55, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0)
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=64, out_features=96, bias=True)
    (5): BatchNorm1d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ELU(alpha=1.0)
    (7): Dropout(p=0.1, inplace=False)
    (8): Line

dbscan tuning

In [8]:
import numpy as np

# extract encoded features
X_train_full_tensor = torch.FloatTensor(X_train_full)
X_train_full_dataset = TensorDataset(X_train_full_tensor)
X_train_full_loader = DataLoader(X_train_full_dataset, batch_size=256)

if use_full_train_set:
    X_encoded = []
    with torch.no_grad():
        for data in X_train_full_loader:
            data_x = data[0].to(device)
            encoded = autoencoder.encode(data_x)
            X_encoded.append(encoded.cpu().numpy())
    X_encoded = np.vstack(X_encoded)

In [9]:
from sklearn.neighbors import NearestNeighbors
from matplotlib import pyplot as plt


def find_eps_range_with_elbow_method(X, k=20, multiplier=(0.5, 2.0), plot=True):
    """
    Find a suitable eps range for DBSCAN using the elbow method.

    Parameters:
    -----------
    X : array-like
        The encoded data points
    k : int, default=20
        Number of neighbors to consider (corresponds to min_samples)
    multiplier : tuple, default=(0.5, 2.0)
        Factors to multiply the elbow point by to create a range
    plot : bool, default=True
        Whether to show the k-distance plot

    Returns:
    --------
    tuple
        (min_eps, max_eps) suitable range for eps parameter
    """

    # Calculate distances to k nearest neighbors for each point
    nbrs = NearestNeighbors(n_neighbors=k, metric="manhattan").fit(X)
    distances, _ = nbrs.kneighbors(X)

    # Sort the distances to the kth neighbor in ascending order
    k_distances = np.sort(distances[:, -1])

    if plot:
        plt.figure(figsize=(10, 6))
        plt.plot(k_distances)
        plt.xlabel("Points (sorted)")
        plt.ylabel(f"Distance to {k}th nearest neighbor")
        plt.title("K-distance Plot for DBSCAN eps Selection")
        plt.grid(True, alpha=0.3)

        # Add horizontal lines at suggested eps range
        # This will be calculated below

        plt.show()

    # Find the elbow point (simple method - you might want a more sophisticated approach)
    # Look for maximum curvature in the sorted distance plot
    n_points = len(k_distances)
    all_coords = np.vstack((range(n_points), k_distances)).T

    # Compute point-to-line distances for all points
    line_vec = all_coords[-1] - all_coords[0]
    line_vec_norm = line_vec / np.sqrt(np.sum(line_vec**2))
    vec_from_first = all_coords - all_coords[0]
    scalar_prod = np.sum(vec_from_first * np.tile(line_vec_norm, (n_points, 1)), axis=1)
    vec_to_line = vec_from_first - np.outer(scalar_prod, line_vec_norm)
    dist_to_line = np.sqrt(np.sum(vec_to_line**2, axis=1))

    # Elbow point is the point with max distance to the line
    elbow_idx = np.argmax(dist_to_line)
    elbow_eps = k_distances[elbow_idx]

    # Create a range around the elbow point
    min_eps = elbow_eps * multiplier[0]
    max_eps = elbow_eps * multiplier[1]

    if plot:
        plt.figure(figsize=(10, 6))
        plt.plot(k_distances)
        plt.axhline(
            y=min_eps,
            color="r",
            linestyle="--",
            alpha=0.7,
            label=f"Min eps: {min_eps:.2f}",
        )
        plt.axhline(
            y=elbow_eps,
            color="g",
            linestyle="-",
            alpha=0.7,
            label=f"Elbow eps: {elbow_eps:.2f}",
        )
        plt.axhline(
            y=max_eps,
            color="r",
            linestyle="--",
            alpha=0.7,
            label=f"Max eps: {max_eps:.2f}",
        )
        plt.axvline(x=elbow_idx, color="g", linestyle=":", alpha=0.5)
        plt.xlabel("Points (sorted)")
        plt.ylabel(f"Distance to {k}th nearest neighbor")
        plt.title("K-distance Plot with Suggested eps Range")
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()

    return min_eps, max_eps

In [10]:
input_dim_encoded = X_encoded.shape[1]

k_for_elbow = int((20 + input_dim_encoded * 2) / 2)
min_eps, max_eps = find_eps_range_with_elbow_method(
    X_encoded,
    k=k_for_elbow,
    plot=False,
)
min_eps, max_eps

(np.float64(5.716840744018555), np.float64(22.86736297607422))

In [ ]:
import optuna
from sklearn.cluster import DBSCAN
from sklearn.metrics import (
    silhouette_score,
    davies_bouldin_score,
    calinski_harabasz_score,
)


def objective_dbscan(
    trial: optuna.Trial,
    X_encoded: np.ndarray,
    evaluation_metric: str = "silhouette",
    eps_range: tuple[float, float] = (0.1, 15.0),
    min_samples_range: tuple[int, int] = (20, 50),
    distance_metric: str = "euclidean",  # "manhattan", "cosine"
    n_jobs: int = -1,
) -> float:
    """
    Inner objective function for optimizing DBSCAN clustering hyperparameters.

    This function is used by Optuna for hyperparameter optimization of a DBSCAN model.
    It evaluates different clustering configurations using the specified evaluation metric.

    Parameters
    ----------
    trial : optuna.Trial
        Optuna trial object used for hyperparameter suggestion
    X_encoded : np.ndarray
        Array of encoded data points to cluster
    evaluation_metric : str, default="silhouette"
        Metric to evaluate cluster quality: "silhouette", "davies_bouldin", or "calinski_harabasz"
    eps_range : tuple[float, float], default=(0.1, 15.0)
        Range (min, max) for the eps parameter of DBSCAN
    min_samples_range : tuple[int, int], default=(20, 50)
        Range (min, max) for the min_samples parameter of DBSCAN
    metric : str, default="euclidean"
        Distance metric for DBSCAN
    score_threshold : float, default=0.60
        Minimum score threshold to consider a clustering configuration valid
    n_jobs : int, default=-1
        Number of parallel jobs to run for DBSCAN

    Returns
    -------
    float
        Clustering quality score (higher is better) unless evaluation_metric is "davies_bouldin"
        Returns -inf for invalid clustering configurations
    """

    def get_score(X, labels, metric_name, mask=None):
        if mask is not None:
            X = X[mask]
            labels = labels[mask]

        if metric_name == "silhouette":
            return silhouette_score(X, labels)
        elif metric_name == "davies_bouldin":
            return -davies_bouldin_score(
                X, labels
            )  # Negative because we want to maximize
        elif metric_name == "calinski_harabasz":
            return calinski_harabasz_score(X, labels)
        else:
            raise ValueError(f"Unknown metric: {metric_name}")

    eps = trial.suggest_float("eps", eps_range[0], eps_range[1])
    min_samples = trial.suggest_int(
        "min_samples", min_samples_range[0], min_samples_range[1]
    )

    dbscan = DBSCAN(
        eps=eps, min_samples=min_samples, metric=distance_metric, n_jobs=n_jobs
    )

    cluster_labels = dbscan.fit_predict(X_encoded)

    # Calculate the number of clusters (excluding noise points)
    unique_clusters = set(cluster_labels)
    n_clusters = len(unique_clusters) - (1 if -1 in unique_clusters else 0)
    print(n_clusters)
    # Set custom attribute for number of clusters
    trial.set_user_attr("n_clusters", n_clusters)

    # Set custom attribute for cluster data points
    cluster_data_points = {}
    for cluster_id in unique_clusters:
        # Store indices of points in each cluster
        cluster_indices = np.where(cluster_labels == cluster_id)[0].tolist()
        cluster_data_points[int(cluster_id)] = len(cluster_indices)
    trial.set_user_attr("cluster_data_points", cluster_data_points)
    print(cluster_data_points)

    if n_clusters < 2:
        print("not enough clusters")
        return -float("inf")  # Penalize solutions with too few clusters

    # For silhouette score, we need to exclude noise points (-1)
    if evaluation_metric == "silhouette":
        mask = cluster_labels != -1
        if sum(mask) < 2:
            print("not enough points in clusters")
            return -float("inf")
        score = get_score(X_encoded, cluster_labels, evaluation_metric, mask)
    else:
        score = get_score(X_encoded, cluster_labels, evaluation_metric)

    return score

/home/jbct/Projects/thesis/db-ocsvm/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
import optuna

dbscan_objective_lambda = lambda trial: objective_dbscan(
    trial,
    X_encoded=X_encoded,
    evaluation_metric=dbscan_tuning_parameters["evaluation_metric"],
    eps_range=(min_eps, max_eps),
    min_samples_range=(1, input_dim_encoded * 2),
    distance_metric=dbscan_tuning_parameters["distance_metric"],
    n_jobs=-1,
)

if with_storage_dbscan:
    dbscan_study = optuna.create_study(
        direction="maximize",
        storage=dbscan_optuna_storage_path,
        study_name="dbscan_study",
        load_if_exists=True,
    )
    dbscan_study.optimize(
        dbscan_objective_lambda,
        n_trials=dbscan_tuning_parameters["trials"],
    )
else:
    dbscan_study = optuna.create_study(direction="maximize")
    dbscan_study.optimize(
        dbscan_objective_lambda,
        n_trials=dbscan_tuning_parameters["trials"],
    )

[I 2025-03-12 04:29:07,767] A new study created in memory with name: no-name-54a9ff14-04da-4b25-9daf-ba3b7d982981
[I 2025-03-12 04:29:07,999] Trial 0 finished with value: 0.5313279628753662 and parameters: {'eps': 19.56210762289765, 'min_samples': 59}. Best is trial 0 with value: 0.5313279628753662.


3
{0: 998, 1: 212, 2: 72, -1: 738}


[I 2025-03-12 04:29:08,218] Trial 1 finished with value: 0.5955008864402771 and parameters: {'eps': 9.648089974241913, 'min_samples': 66}. Best is trial 1 with value: 0.5955008864402771.


2
{0: 912, 1: 138, -1: 970}


[I 2025-03-12 04:29:08,486] Trial 2 finished with value: 0.5866093635559082 and parameters: {'eps': 12.407587636002894, 'min_samples': 98}. Best is trial 1 with value: 0.5955008864402771.


...

2
{0: 912, 1: 132, -1: 976}


[I 2025-03-12 04:29:09,941] Trial 8 finished with value: 0.5499024987220764 and parameters: {'eps': 22.115938462489606, 'min_samples': 94}. Best is trial 6 with value: 0.6009919047355652.


2
{0: 1019, 1: 214, -1: 787}


[I 2025-03-12 04:29:10,226] Trial 9 finished with value: 0.4436343014240265 and parameters: {'eps': 16.22089235907331, 'min_samples': 3}. Best is trial 6 with value: 0.6009919047355652.


39
{0: 100, 1: 1007, 2: 82, 3: 3, 4: 19, 5: 3, 6: 215, 7: 56, 8: 17, 9: 32, 10: 6, 11: 94, 12: 12, 13: 3, 14: 8, 15: 9, 16: 13, 17: 35, 18: 4, 19: 10, 20: 8, 21: 12, 22: 7, 23: 3, 24: 3, 25: 4, 26: 3, 27: 7, 28: 4, 29: 3, 30: 13, 31: 8, 32: 3, 33: 4, 34: 3, 35: 3, 36: 5, 37: 4, 38: 3, -1: 192}


In [13]:
import pprint

# get dbscan best parameters
eps = dbscan_study.best_params["eps"]
min_samples = dbscan_study.best_params["min_samples"]

# get dbscan best trial
best_trial_dbscan = dbscan_study.best_trial
best_trial_dbscan_user_attrs = best_trial_dbscan.user_attrs

n_clusters = best_trial_dbscan_user_attrs["n_clusters"]
cluster_data_points = best_trial_dbscan_user_attrs["cluster_data_points"]

print(f"eps = {eps}")
print(f"min_samples = {min_samples}")
print(f"n_clusters = {n_clusters}")
print("cluster_data_points")
pprint.pprint(cluster_data_points)

eps = 7.745111403504765
min_samples = 40
n_clusters = 2
cluster_data_points
{-1: 995, 0: 890, 1: 135}


fit the DBSCAN

In [14]:
class DBOCSVM_V2:
    def __init__(
        self,
        kernel="rbf",
        gamma="scale",
        nu=0.5,
        eps=0.5,
        min_samples=10,
        tree_metric="euclidean",
        dbscan_metric="euclidean",
        algorithm="kd_tree",  # or 'ball_tree'
        n_jobs=-1,  # Add n_jobs parameter
    ):
        self.kernel = kernel
        self.gamma = gamma
        self.nu = nu
        self.algorithm = algorithm
        self.tree_metric = tree_metric
        self.dbscan_metric = dbscan_metric
        self.dbscan = DBSCAN(
            eps=eps, min_samples=min_samples, n_jobs=n_jobs, metric=dbscan_metric
        )  # Make it so that it can accept a metric parameter
        self.svms = {}  # One SVM per cluster
        self.dbscan_centroids = {}  # To store cluster centroids
        self.cluster_points = {}  # Store points in each cluster
        self.tree = None
        # These attributes are mainly used for inspection purposes
        self.cluster_sizes = {}  # Number of points in each cluster
        self.n_jobs = n_jobs  # Store n_jobs
        self.cluster_labels = None
        self.unique_clusters = None

    def fit_cluster(
        self,
        X,
        verbose=False,
    ):
        """
        Parameters:
        -----------
        X : array-like
            Training data
        """

        X = X.values if isinstance(X, pd.DataFrame) else X

        """
        NOTE: Current DBSCAN only uses euclidean distance, so the metric parameter is not used
        TODO: Add metric parameter to DBSCAN to handle different distance metrics
        'euclidean': Standard Euclidean distance. This is the default metric.
        'manhattan': Manhattan or L1 distance (sum of absolute differences).
        'chebyshev': Chebyshev or maximum distance.
        'minkowski': Minkowski distance, a generalization of Euclidean and Manhattan distance. The power parameter p of the Minkowski metric can be controlled by the p parameter of DBSCAN.
        'wminkowski': Weighted Minkowski distance.
        'seuclidean': Standardized Euclidean distance.
        'mahalanobis': Mahalanobis distance.
        """

        if verbose:
            print("Fitting DBSCAN...")
        # NOTE: we use the dbscan that was initialized in the constructor
        self.cluster_labels = self.dbscan.fit_predict(X)

        if verbose:
            print("DBSCAN Fitted...")

        self.unique_clusters = np.unique(self.cluster_labels)

        if verbose:
            print(f"Unique Clusters: {self.unique_clusters}")

        for cluster in self.unique_clusters:
            n_points = np.sum(self.cluster_labels == cluster)
            self.cluster_sizes[int(cluster)] = int(n_points)

        if verbose:
            print(f"Cluster Sizes: {self.cluster_sizes}")

    def fit_ocsvm(
        self,
        X,
        parameter_list=None,
        verbose=False,
    ):
        """
        Parameters:
        -----------
        X : array-like
            Training data
        parameter_list: dictionary of dictionaries
            Each key in the dictionary is the cluster number and
            the value is a dictionary containing the parameters for OCSVM
            each dictionary looks like this:
            {
                0 : {
                kernel: rbf, linear, poly, or sigmoid,
                gamma: 'scale', 'auto' or a float,
                nu: a float between 0 and 1 e.g 0.2,
                }
            }
        """
        X = X.values if isinstance(X, pd.DataFrame) else X

        if parameter_list is None:
            raise ValueError("parameter_list cannot be None")

        if len(parameter_list) < len(self.unique_clusters) - 1:
            raise ValueError(
                "Number of parameters should be equal or greater than the number of clusters"
            )

        def filter_dict(original_dict, keys_to_keep):
            return {k: original_dict[k] for k in keys_to_keep if k in original_dict}

        if len(parameter_list) >= len(self.unique_clusters) - 1:
            cluster_count = list(self.cluster_sizes.keys())
            if -1 in cluster_count:
                cluster_count.remove(-1)
            cluster_count

            parameter_list = filter_dict(parameter_list, cluster_count)

        for cluster in self.unique_clusters:

            if cluster == -1:  # Skip noise cluster for SVM training
                continue

            if verbose:
                print(
                    f"Training for cluster {cluster} with {self.cluster_sizes[cluster]} points"
                )

            # Boolean masking to get points in the current cluster
            points = X[self.cluster_labels == cluster]
            self.cluster_points[cluster] = points

            if len(points) > 0:
                # use parameters defined in constructor if not provided
                if parameter_list is None:
                    ocsvm = OneClassSVM(
                        kernel=self.kernel,
                        nu=self.nu,
                        gamma=self.gamma,
                    )
                else:
                    ocsvm = OneClassSVM(
                        kernel=parameter_list[cluster]["kernel"],
                        nu=parameter_list[cluster]["nu"],
                        gamma=parameter_list[cluster]["gamma"],
                    )
                    if verbose:
                        print(
                            f"OCSVM for cluster {cluster} uses nu: {parameter_list[cluster]['nu']}, gamma: {parameter_list[cluster]['gamma']}, kernel: {parameter_list[cluster]['kernel']}"
                        )

                ocsvm.fit(points)

                self.svms[cluster] = ocsvm

                """
                TODO: Explore other alternatives for centroid calculation
                "->" means the following line might be a downside of the current approach.
                
                - Median: More robust to outliers than the mean (`np.median(points, axis=0)`).
                    -> Less representative if data is asymmetric  
                - Trimmed Mean: Removes extreme values before computing the mean (`scipy.stats.trim_mean`).
                    ->   Requires choosing the trimming percentage
                - Weighted Mean: Assigns importance to points based on reliability.  
                    ->  Requires defining weights
                - Geometric Median: Minimizes sum of distances to all points. More robust to outliers than the mean.
                    -> computationally expensive (`scipy.spatial`)
                - Distance Metrics: Use median for Manhattan distance and mean for Euclidean distance.
                    -> Requires choosing the distance metric
                    
                """
                self.dbscan_centroids[cluster] = np.mean(points, axis=0)

        # Build tree with cluster centroids
        centroids = [self.dbscan_centroids[c] for c in self.dbscan_centroids if c != -1]
        self.valid_clusters = list(self.dbscan_centroids.keys())
        if len(centroids) > 0:
            centroids = np.array(centroids)
            if self.algorithm == "kd_tree":
                self.tree = KDTree(centroids, metric=self.tree_metric)
            elif self.algorithm == "ball_tree":
                self.tree = BallTree(centroids, metric=self.tree_metric)

    def fit(
        self,
        X,
        dbscan_evaluation_metric="silhouette",  # only used for reruns
        dbscan_rerun=False,  # only used for reruns
        dbscan_rerun_trials=10,  # only used for reruns
        parameter_list=None,
        verbose=False,
    ):
        """
        Parameters:
        -----------
        X : array-like
            Training data
        dbscan_evaluation_metric : str
            Metric to optimize ('silhouette', 'davies_bouldin', or 'calinski_harabasz')
        dbscan_rerun : bool
            Whether to rerun DBSCAN after fitting the model with the best parameters
        dbscan_rerun_trials : int
            Number of reruns for DBSCAN after fitting the model with the best parameters
        parameter_list: dictionary of dictionaries
            Each key in the dictionary is the cluster number and
            the value is a dictionary containing the parameters for OCSVM
            each dictionary looks like this:
            {
                0 : {
                kernel: rbf, linear, poly, or sigmoid,
                gamma: 'scale', 'auto' or a float,
                nu: a float between 0 and 1 e.g 0.2,
                }
            }
        """

        X = X.values if isinstance(X, pd.DataFrame) else X

        """
        NOTE: Current DBSCAN only uses euclidean distance, so the metric parameter is not used
        TODO: Add metric parameter to DBSCAN to handle different distance metrics
        'euclidean': Standard Euclidean distance. This is the default metric.
        'manhattan': Manhattan or L1 distance (sum of absolute differences).
        'chebyshev': Chebyshev or maximum distance.
        'minkowski': Minkowski distance, a generalization of Euclidean and Manhattan distance. The power parameter p of the Minkowski metric can be controlled by the p parameter of DBSCAN.
        'wminkowski': Weighted Minkowski distance.
        'seuclidean': Standardized Euclidean distance.
        'mahalanobis': Mahalanobis distance.
        """
        if verbose:
            print("Fitting DBSCAN...")
        # NOTE: we use the dbscan that was initialized in the constructor
        cluster_labels = self.dbscan.fit_predict(X)
        if verbose:
            print("DBSCAN Fitted...")

        if dbscan_rerun:
            if verbose:
                print("Rerunning DBSCAN...")

            if dbscan_evaluation_metric == "silhouette":
                current_score = silhouette_score(X, cluster_labels)
            elif dbscan_evaluation_metric == "davies_bouldin":
                current_score = davies_bouldin_score(X, cluster_labels)
            else:  # calinski_harabasz
                current_score = calinski_harabasz_score(X, cluster_labels)

            for i in range(dbscan_rerun_trials):
                if verbose:
                    print(f"DBSCAN Rerun {i+1}...")

                new_cluster_labels = self.dbscan.fit_predict(X)

                if dbscan_evaluation_metric == "silhouette":
                    new_score = silhouette_score(X, new_cluster_labels)
                    if new_score > current_score:
                        cluster_labels = new_cluster_labels
                        current_score = new_score
                elif dbscan_evaluation_metric == "davies_bouldin":
                    new_score = davies_bouldin_score(X, new_cluster_labels)
                    if new_score < current_score:
                        cluster_labels = new_cluster_labels
                        current_score = new_score
                else:  # calinski_harabasz
                    new_score = calinski_harabasz_score(X, new_cluster_labels)
                    if new_score > current_score:
                        cluster_labels = new_cluster_labels
                        current_score = new_score

        unique_clusters = np.unique(cluster_labels)

        if verbose:
            print(f"Unique Clusters: {unique_clusters}")

        for cluster in unique_clusters:
            # Store the number of points in the cluster
            # mainly for inspection purposes
            n_points = np.sum(cluster_labels == cluster)
            self.cluster_sizes[int(cluster)] = int(n_points)

        if verbose:
            print(f"Cluster Sizes: {self.cluster_sizes}")

        if parameter_list is not None and (len(parameter_list)) < (
            len(unique_clusters) - 1
        ):
            raise ValueError(
                "Number of parameters should be equal or greater than the number of clusters"
            )

        def filter_dict(original_dict, keys_to_keep):
            return {k: original_dict[k] for k in keys_to_keep if k in original_dict}

        if parameter_list is not None and (len(parameter_list)) >= (
            len(unique_clusters) - 1
        ):
            cluster_count = list(self.cluster_sizes.keys())
            cluster_count.remove(-1)
            cluster_count

            parameter_list = filter_dict(parameter_list, cluster_count)

        self.parameter_list = parameter_list

        for cluster in unique_clusters:

            # Store the number of points in the cluster
            # n_points = np.sum(cluster_labels == cluster)
            # self.cluster_sizes[int(cluster)] = int(n_points)

            if cluster == -1:  # Skip noise cluster for SVM training
                continue

            if verbose:
                print(
                    f"Training for cluster {cluster} with {self.cluster_sizes[cluster]} points"
                )

            # Boolean masking to get points in the current cluster
            points = X[cluster_labels == cluster]
            self.cluster_points[cluster] = points

            if len(points) > 0:
                # use parameters defined in constructor if not provided
                if parameter_list is None:
                    ocsvm = OneClassSVM(
                        kernel=self.kernel,
                        nu=self.nu,
                        gamma=self.gamma,
                        degree=self.degree,
                        coef0=self.coef0,
                        tol=self.tol,
                        shrinking=self.shrinking,
                        cache_size=self.cache_size,
                        max_iter=self.max_iter,
                    )
                else:
                    ocsvm = OneClassSVM(
                        kernel=parameter_list[cluster]["kernel"],
                        nu=parameter_list[cluster]["nu"],
                        gamma=parameter_list[cluster]["gamma"],
                        degree=self.degree,
                        coef0=self.coef0,
                        tol=self.tol,
                        shrinking=self.shrinking,
                        cache_size=self.cache_size,
                        max_iter=self.max_iter,
                    )
                    if verbose:
                        print(
                            f"OCSVM for cluster {cluster} uses nu: {parameter_list[cluster]['nu']}, gamma: {parameter_list[cluster]['gamma']}, kernel: {parameter_list[cluster]['kernel']}"
                        )
                ocsvm.fit(points)

                self.svms[cluster] = ocsvm

                """
                TODO: Explore other alternatives for centroid calculation
                "->" means the following line might be a downside of the current approach.
                
                - Median: More robust to outliers than the mean (`np.median(points, axis=0)`).
                    -> Less representative if data is asymmetric  
                - Trimmed Mean: Removes extreme values before computing the mean (`scipy.stats.trim_mean`).
                    ->   Requires choosing the trimming percentage
                - Weighted Mean: Assigns importance to points based on reliability.  
                    ->  Requires defining weights
                - Geometric Median: Minimizes sum of distances to all points. More robust to outliers than the mean.
                    -> computationally expensive (`scipy.spatial`)
                - Distance Metrics: Use median for Manhattan distance and mean for Euclidean distance.
                    -> Requires choosing the distance metric
                    
                """
                self.dbscan_centroids[cluster] = np.mean(points, axis=0)

        # Build tree with cluster centroids
        centroids = [self.dbscan_centroids[c] for c in self.dbscan_centroids if c != -1]
        self.valid_clusters = list(self.dbscan_centroids.keys())
        if len(centroids) > 0:
            centroids = np.array(centroids)
            if self.algorithm == "kd_tree":
                self.tree = KDTree(
                    centroids, leaf_size=self.leaf_size, metric=self.tree_metric
                )
            elif self.algorithm == "ball_tree":
                self.tree = BallTree(
                    centroids, leaf_size=self.leaf_size, metric=self.tree_metric
                )

    def predict(self, X):
        predictions = np.ones(len(X))
        X = X.values if isinstance(X, pd.DataFrame) else X

        if self.tree is None:
            return -1 * np.ones(len(X))

        # Find nearest centroid
        dist, ind = self.tree.query(X, k=1)
        nearest_clusters = [self.valid_clusters[i] for i in ind.flatten()]

        for i, cluster in enumerate(nearest_clusters):
            if cluster in self.svms:
                predictions[i] = self.svms[cluster].predict([X[i]])[0]
            else:
                predictions[i] = -1  # Anomaly if no SVM for cluster

        return predictions

In [15]:
# Create DB-OC-SVM model with default ocsvm parameters
dbocsvm = DBOCSVM_V2(
    kernel="rbf",
    gamma="auto",
    nu=0.2,
    eps=eps,
    min_samples=min_samples,
    dbscan_metric=dbscan_tuning_parameters["distance_metric"],
    algorithm="kd_tree",  # ball_tree, kd_tree,
)

In [16]:
dbocsvm.fit_cluster(X_encoded, verbose=True)

Fitting DBSCAN...
DBSCAN Fitted...
Unique Clusters: [-1  0  1]
Cluster Sizes: {-1: 995, 0: 890, 1: 135}


importing test set

In [17]:
test_df = pd.read_csv(test_set_path)
print(test_df.shape)
test_df.head(1)

(22543, 125)


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,attack_binary,attack_categorical,attack_class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,neptune,DoS


In [18]:
# Splitting into X and y
X_test = test_df.drop(
    columns=["attack_binary", "attack_categorical", "attack_class"]
).values
y_test = test_df["attack_binary"].values
y_test_class = test_df["attack_class"]

print(X_test.shape, y_test.shape)

(22543, 122) (22543,)


reconstruction error inspection

In [19]:
# Separate normal and anomaly samples from test set
X_test_normal = X_test[y_test == 1]
X_test_anomaly = X_test[y_test == -1]

print(f"Normal test samples: {X_test_normal.shape[0]}")
print(f"Anomaly test samples: {X_test_anomaly.shape[0]}")

# Convert test data to PyTorch tensors
X_test_normal_tensor = torch.FloatTensor(X_test_normal).to(device)
X_test_anomaly_tensor = torch.FloatTensor(X_test_anomaly).to(device)

# Create DataLoaders for test data evaluation
normal_test_dataset = TensorDataset(X_test_normal_tensor)
anomaly_test_dataset = TensorDataset(X_test_anomaly_tensor)
normal_test_loader = DataLoader(normal_test_dataset, batch_size=256, shuffle=False)
anomaly_test_loader = DataLoader(anomaly_test_dataset, batch_size=256, shuffle=False)


def calculate_reconstruction_error(model, loader):
    model.eval()
    total_loss = 0
    total_samples = 0
    criterion = nn.MSELoss(reduction="none")

    with torch.no_grad():
        for batch in loader:
            x = batch[0]
            outputs = model(x)
            # Calculate MSE for each sample
            loss = criterion(outputs, x)
            loss = loss.mean(dim=1)
            total_loss += torch.sum(loss).item()
            total_samples += x.size(0)

    return total_loss / total_samples


# Function to evaluate a model's reconstruction performance
def evaluate_model(model):
    normal_loss = calculate_reconstruction_error(model, normal_test_loader)
    anomaly_loss = calculate_reconstruction_error(model, anomaly_test_loader)
    loss_difference = anomaly_loss - normal_loss

    return {
        "normal_loss": normal_loss,
        "anomaly_loss": anomaly_loss,
        "loss_difference": loss_difference,
    }


reconstruction_error = evaluate_model(autoencoder)
reconstruction_error

Normal test samples: 9711
Anomaly test samples: 12832


{'normal_loss': 0.000964479972240596,
 'anomaly_loss': 0.010680428181224482,
 'loss_difference': 0.009715948208983886}

best parameters and values

In [20]:
autoencoder_architecture = {
    "input_dim": existing_model_architecture["input_dim"],
    "hidden_dims": existing_model_architecture["hidden_dims"],
    "latent_dim": existing_model_architecture["latent_dim"],
    "activation_type": existing_model_architecture["activation_type"],
    "negative_slope": existing_model_architecture["negative_slope"],
    "dropout_rate": existing_model_architecture["dropout_rate"],
    "output_activation_type": existing_model_architecture["output_activation_type"],
    "val_loss": checkpoint["val_loss"],
}

print("Best autoencoder model:")
pprint.pprint(autoencoder_architecture, sort_dicts=False)
print("")

print("Reconstruction error:")
pprint.pprint(reconstruction_error, sort_dicts=False)
print("")

best_dbscan_parameters = {
    "eps": eps,
    "min_samples": min_samples,
    "distance_metric": dbscan_tuning_parameters["distance_metric"],
    "evaluation_metric": dbscan_tuning_parameters["evaluation_metric"],
    "score": best_trial_dbscan.value,
    "n_clusters": n_clusters,
    "cluster_data_points": cluster_data_points,
}

print("Best dbscan parameters")
pprint.pprint(best_dbscan_parameters, sort_dicts=False)
print("")

Best autoencoder model:
{'input_dim': 122,
 'hidden_dims': [96, 64],
 'latent_dim': 55,
 'activation_type': 'ELU',
 'negative_slope': 0.02,
 'dropout_rate': 0.1,
 'output_activation_type': 'Sigmoid',
 'val_loss': 0.00011326836558407006}

Reconstruction error:
{'normal_loss': 0.000964479972240596,
 'anomaly_loss': 0.010680428181224482,
 'loss_difference': 0.009715948208983886}

Best dbscan parameters
{'eps': 7.745111403504765,
 'min_samples': 40,
 'distance_metric': 'manhattan',
 'evaluation_metric': 'silhouette',
 'score': 0.6009919047355652,
 'n_clusters': 2,
 'cluster_data_points': {0: 890, 1: 135, -1: 995}}



In [21]:
import json

tuning_result = {
    "autoencoder": autoencoder_architecture,
    "reconstruction_error": reconstruction_error,
    "dbscan": best_dbscan_parameters,
}

results = {
    "max_score": 0,
    "tuning_results": {},
}

os.makedirs("tuning_results", exist_ok=True)
if os.path.exists(results_path):
    with open(results_path, "r") as file:
        existing_results = json.load(file)
        if existing_results["max_score"] < best_trial_dbscan.value:
            with open(results_path, "w") as f:
                existing_results["max_score"] = best_dbscan_parameters["score"]
                tuning_result_id = len(existing_results["tuning_results"])
                existing_results["tuning_results"][tuning_result_id] = tuning_result
                json.dump(existing_results, f)
else:
    with open(results_path, "w") as f:
        results["max_score"] = best_dbscan_parameters["score"]
        results["tuning_results"][0] = tuning_result
        json.dump(results, f)